In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import VGG16
from keras_tuner import HyperModel, GridSearch
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
training_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/Front Off',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

Found 318 images belonging to 3 classes.


In [ ]:
test_set = datagen.flow_from_directory(
    '/content/drive/MyDrive/Dataset/Front Off',
    target_size=(256, 256),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 79 images belonging to 3 classes.


In [ ]:
class VGGHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=self.input_shape)

        # Add custom layers
        x = Flatten()(base_model.output)
        x = Dense(units=hp.Choice('dense_units', [128, 192, 256, 320, 384, 448, 512]), activation='relu')(x)
        x = Dropout(rate=hp.Choice('dropout_rate', [0.3, 0.4, 0.5, 0.6, 0.7]))(x)
        predictions = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=predictions)

        # Freeze the base model layers
        for layer in base_model.layers:
            layer.trainable = False

        # Compile the model
        model.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

In [ ]:
hypermodel = VGGHyperModel(input_shape=(256, 256, 3), num_classes=3)

In [ ]:
tuner = GridSearch(
    hypermodel,
    objective='val_accuracy',
    directory='vgg_hyperparameter_tuning',
    project_name='vgg16_grid_tuning'
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
tuner.search(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=10,
    validation_steps=80
)

Trial 70 Complete [00h 03m 14s]
val_accuracy: 0.5696202516555786

Best val_accuracy So Far: 0.607594907283783
Total elapsed time: 04h 40m 24s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
print(f"Best Dense Units: {best_hps.get('dense_units')}")
print(f"Best Dropout Rate: {best_hps.get('dropout_rate')}")
print(f"Best Optimizer: {best_hps.get('optimizer')}")

Best Dense Units: 320
Best Dropout Rate: 0.3
Best Optimizer: adam


In [ ]:
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(
    training_set,
    steps_per_epoch=10,
    epochs=20,
    validation_data=test_set,
    validation_steps=80
)

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.4070 - loss: 7.0528 - val_accuracy: 0.5696 - val_loss: 3.1263
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.3259 - loss: 3.5367 - val_accuracy: 0.4304 - val_loss: 1.2373
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 617ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.3671 - val_loss: 1.2552
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.4385 - loss: 1.5717 - val_accuracy: 0.4304 - val_loss: 1.2084
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5085 - loss: 1.0699 - val_accuracy: 0.5443 - val_loss: 1.1728
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.5823 - val_loss: 1.1681
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.6279 - loss: 0.8298 - val_accuracy: 0.4937 - val_loss: 1.0850
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 866ms/step - accuracy: 0.5491 - loss: 0.9884 - val_accuracy:

In [ ]:
final_scores = best_model.evaluate(test_set)
print(f"Final Test Accuracy: {final_scores[1]}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.4934 - loss: 1.2000
Final Test Accuracy: 0.5063291192054749


In [ ]:
y_pred = best_model.predict(test_set)
y_true = test_set.labels
y_pred_classes = np.argmax(y_pred, axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step


In [ ]:
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

In [ ]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.5684695051783659
Recall: 0.5569620253164557
F1 Score: 0.48750731131294617


In [ ]:
best_model.save('best_modelfrontonVGG.keras')